In [1]:
import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline
import time
from torch.autograd import Variable
import pandas as pd
import seaborn

In [3]:
def conv_block(in_channels, out_channels, pool=False, drop=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    if drop: layers.append(nn.Dropout())
    return nn.Sequential(*layers)

In [4]:
class SignConvNet(nn.Module):
    def __init__(self, in_channels, out_classes):
        super().__init__()
        self.conv1 = conv_block(in_channels, 16)
        self.conv2 = conv_block(16, 32, pool=True)
        self.conv3 = conv_block(32, 64, pool=True, drop=True)
        self.fc =  nn.Sequential(*[
                        nn.Flatten(),
                        nn.Linear(7 * 7 * 64, out_classes)
                    ])
        
    def forward(self, img):
        img = self.conv1(img)
        img = self.conv2(img)
        img = self.conv3(img)
        return self.fc(img)

In [5]:
# Класс детектирования и обработки лица с веб-камеры 
class FaceDetector(object):

    def __init__(self, mtcnn, hand_detector, hand_classifier, channels=1):
        # Создаем объект для считывания потока с веб-камеры(обычно вебкамера идет под номером 0. иногда 1)
        self.cap = cv2.VideoCapture(0) 
        self.mtcnn = mtcnn
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.hand_detector = hand_detector.to(self.device)
        self.hand_classifier = hand_classifier.to(self.device)
        self.channels = channels

    # Функция рисования найденных параметров на кадре
    def _draw(self, frame, boxes, probs, col, title = ''):
        try:
            for box, prob in zip(boxes, probs):
            # Рисуем обрамляющий прямоугольник объекта на кадре
                cv2.rectangle(frame,
                              (int(box[0]), int(box[1])),
                              (int(box[2]), int(box[3])),
                              col, thickness=2)
                if title > '':
                    # пишем на кадре класс
                    cv2.putText(frame, 
                        title, (int(box[2]), int(box[3])), cv2.FONT_HERSHEY_SIMPLEX, 1, col, 2, cv2.LINE_AA)

#                 # Рисуем особенные точки
#                 cv2.circle(frame, (int(ld[0][0]),int(ld[0][1])), 5, (0, 0, 255), -1)
#                 cv2.circle(frame, (int(ld[1][0]),int(ld[1][1])), 5, (0, 0, 255), -1)
#                 cv2.circle(frame, (int(ld[2][0]),int(ld[2][1])), 5, (0, 0, 255), -1)
#                 cv2.circle(frame, (int(ld[3][0]),int(ld[3][1])), 5, (0, 0, 255), -1)
#                 cv2.circle(frame, (int(ld[4][0]),int(ld[4][1])), 5, (0, 0, 255), -1)
        except Exception as e:
            print('Something wrong im draw function!')
            print(f'error : {e}')

        return frame
    
    # Функция для вырезания объектов с кадра
    @staticmethod
    def crop_faces(frame, boxes):
        faces = []
        for i, box in enumerate(boxes):
            faces.append(frame[int(box[1]-40):int(box[3]+40), 
                int(box[0]-40):int(box[2]+40)])
        return faces
    
    @staticmethod
    def digit_to_classname(digit):
        if digit == 0:
            return 'A'
        elif digit == 1:
            return 'B'
        elif digit == 2:
            return 'C'
        elif digit == 3:
            return 'D'
        elif digit == 4:
            return 'E'
        elif digit == 5:
            return 'F'
        elif digit == 6:
            return 'G'
        elif digit == 7:
            return 'H'
        elif digit == 8:
            return 'I'
        elif digit == 9:
            return 'J'
        elif digit == 10:
            return 'K'
        elif digit == 11:
            return 'L'
        elif digit == 12:
            return 'M'
        elif digit == 13:
            return 'N'
        elif digit == 14:
            return 'O'
        elif digit == 15:
            return 'P'
        elif digit == 16:
            return 'Q'
        elif digit == 17:
            return 'R'
        elif digit == 18:
            return 'S'
        elif digit == 19:
            return 'T'
        elif digit == 20:
            return 'U'
        elif digit == 21:
            return 'V'
        elif digit == 22:
            return 'W'
        elif digit == 23:
            return 'X'
        elif digit == 24:
            return 'Y'
        elif digit == 25:
            return 'Z'

       
    # Функция в которой будет происходить процесс считывания и обработки каждого кадра
    def run(self):              
        # Заходим в бесконечный цикл
        while True:
            # Считываем каждый новый кадр - frame
            # ret - логическая переменая. Смысл - считали ли мы кадр с потока или нет
            ret, frame = self.cap.read()

            try:
                # детектируем расположение лица на кадре, вероятности на сколько это лицо
                # и особенные точки лица
                f_boxes, f_probs, f_landmarks = self.mtcnn.detect(frame, landmarks=True)
                if f_boxes is not None: 
                    # Рисуем на кадре лицо
                    self._draw(frame, f_boxes, f_probs, (255, 0, 0), title = '', )
                    
                    # если лицо нашлось и перед камерой человек, то
                    # детектируем расположение кисти на кадре
                    frame_frcnn = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)/255
                    frame_frcnn = cv2.resize(frame_frcnn, (596, 400))
                    frame_frcnn = np.asarray(frame_frcnn).astype('float')
                    frame_frcnn = torch.as_tensor(frame_frcnn)
                    frame_frcnn = frame_frcnn.permute(2, 0, 1)
                    frame_frcnn = frame_frcnn.unsqueeze(0).to(self.device).float()
                    h_outputs = self.hand_detector(frame_frcnn)
                    h_outputs = [{k: v.to(self.device) for k, v in t.items()} for t in h_outputs]
                    h_boxes = h_outputs[0]['boxes']
                    h_scores = h_outputs[0]['scores']
        #             print('h ', h_boxes)

                    # если кисть нашлась:
                    if h_boxes.shape[0] > 0:
                        h_boxes = h_boxes.data.cpu().numpy().astype(int)
                        print('hb ', h_boxes)
                        # Вырезаем руку из кадра
                        hands = self.crop_faces(frame, h_boxes)
                        cropped_hand = hands[0]
                        # Меняем размер изображения для входа в классификатор
                        cropped_hand = cv2.resize(cropped_hand, (28,28))
                        hand = cv2.cvtColor(cropped_hand, cv2.COLOR_BGR2RGB)
                        # Превращаем в 1-канальное серое изображение
                        hand = cv2.cvtColor(hand, cv2.COLOR_BGR2GRAY)
                        # Далее мы подготавливаем наш кадр для считывания нс
                        hand = Image.fromarray(hand)
                        hand = np.asarray(hand).astype('float')
                        hand = torch.as_tensor(hand)
                        torch_hand = hand.unsqueeze(0).to(self.device).float()
                        # и закидываем в классификатор
                        output = self.hand_classifier(torch_hand[None, ...])
                        _, test_pred = torch.max(output, dim = 1)
                        # Интерпретируем предсказание как строку нашей эмоции
                        letter = self.digit_to_classname(test_pred[0])
                        print(letter)

                        # Рисуем на кадре руку
                        self._draw(frame, h_boxes, h_scores, (0, 0, 255), title = letter)

#                 cv2.imshow("Gray face", hand_img)

            except Exception as e:
                print('Something wrong im main cycle!')
                print(f'error : {e}')

            # Показываем кадр в окне, и назвываем его(окно) - 'Face Detection'
            cv2.imshow('Face Detection', frame)
            
            
            # Функция, которая проверяет нажатие на клавишу 'q'
            # Если нажатие произошло - выход из цикла. Конец работы приложения
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        # Очищаем все объекты opencv, что мы создали
        self.cap.release()
        cv2.destroyAllWindows()

In [6]:
# face detector
mtcnn = MTCNN()

# hand detector
hand_detector = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
in_features = hand_detector.roi_heads.box_predictor.cls_score.in_features
hand_detector.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
hand_detector.load_state_dict(torch.load('./frcnn400_1.pth'))
hand_detector.eval()

# hand sign classifier
hand_classifier = SignConvNet(1, 26)
hand_classifier.load_state_dict(torch.load('./sign_conv.pth'))
hand_classifier.eval()


SignConvNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.5, inplace=False)
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=26, bias=True)


In [7]:
# Создаем объект нашего класса приложения
fcd = FaceDetector(mtcnn, hand_detector, hand_classifier)
# Запускаемq
fcd.run()

hb  [[ 90 260 232 378]]
T
hb  [[235 281 254 301]]
Y
hb  [[149 201 264 333]]
L
hb  [[162 240 202 306]]
L
hb  [[164 239 207 307]]
L
hb  [[164 242 205 305]]
L
hb  [[161 239 203 307]]
L
hb  [[162 240 201 306]]
L
hb  [[159 235 206 299]]
L
